## AgentOps Integration Guide - Step by Step
### Overview
This notebook demonstrates how to integrate AgentOps with Google's Gemini 1.5 Flash API to track and monitor AI agent interactions. AgentOps provides observability for AI agents, allowing you to monitor performance, costs, and agent behaviors.

## What is AgentOps?
AgentOps is a comprehensive monitoring and observability platform specifically designed for AI agents and multi-agent systems. It acts as a "mission control" for your AI applications, providing real-time insights into how your agents are performing, what they're doing, and how much they're costing you.

## Why Use AgentOps?
Without AgentOps, when you deploy AI agents, you're essentially flying blind:

* ❌ No visibility into agent decision-making processes
* ❌ No cost tracking across multiple API calls
* ❌ Difficult to debug when agents fail or behave unexpectedly
* ❌ No performance metrics to optimize your agents
* ❌ Hard to track which agent did what in multi-agent scenarios

With AgentOps, you get complete visibility:

* ✅ Real-time Monitoring: See exactly what your agents are doing as they do it
* ✅ Cost Tracking: Monitor token usage and costs across all your AI interactions
* ✅ Performance Analytics: Track response times, success rates, and error patterns
* ✅ Agent Attribution: Know which specific agent performed each action
* ✅ Session Management: Group related agent activities for easier analysis
* ✅ Debugging Tools: Detailed logs and traces for troubleshooting

## Step 1: Installation and Dependencies

In [1]:
!pip install agentops python-dotenv google-generativeai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.3/285.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 23.5 MB/s eta 0:00:00


### Purpose: Install the required packages:
- agentops: The main monitoring and observability library
- google-generativeai: Google's official Gemini Python client
- python-dotenv: For loading environment variables from .env files

## Step 2: Import Required Libraries

In [2]:
import agentops
from agentops.sdk.decorators import agent, operation
import google.generativeai as genai
import os
from dotenv import load_dotenv
import logging
from IPython.display import display, Markdown

## Step 3: Environment Setup and Configuration

In [3]:
load_dotenv()
GEMINI_API_KEY = "your_Gemini_api_key"
AGENTOPS_API_KEY = "your_agentops_api_key"
logging.basicConfig(
    level=logging.DEBUG
)

## Step 4: Initialize Services

In [8]:
gemini_model = genai.GenerativeModel('gemini-1.5-flash')

# Initialize AgentOps
agentops.init(AGENTOPS_API_KEY, tags=["gemini-1.5-flash-notebook"])
print("✅ Setup complete!")

✅ Setup complete!


## Step 5: Basic Gemini API Call with Monitoring

In [10]:
genai.configure(api_key=GEMINI_API_KEY)

# Simple API call with AgentOps monitoring
prompt = "Write a 12 word poem about secret agents."

# Generate response with Gemini 1.5 Flash
response = gemini_model.generate_content(
    prompt,
    generation_config=genai.types.GenerationConfig(
        temperature=0.5,
    )
)

print(response.text)

Shadows dance, secrets kept,
Missions bold, lives swept,
Danger near, truth unclear.



## Step 6: End Session

In [11]:
agentops.end_session("Success")

🖇 AgentOps: end_session() is deprecated and will be removed in v4 in the future. Use agentops.end_trace() instead.
🖇 AgentOps: end_session called but no active trace context found.


## Step 7: Multi-Agent Architecture Setup
### Initialize New Session

In [12]:
# Reinitialize for multi-agent demo
agentops.init(AGENTOPS_API_KEY, tags=["multi-agent-gemini-notebook"])
gemini_client = genai.GenerativeModel('gemini-1.5-flash')

## Create QA Agent Class

In [13]:
@agent(name="qa")
class QaAgent:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-1.5-flash')

    @operation
    def completion(self, prompt: str):
        system_prompt = """You are a QA engineer. Your task is to write Python unit tests.
        Only output Python code without any markdown formatting or code blocks.
        Focus on comprehensive test coverage."""

        full_prompt = f"{system_prompt}\n\nUser request: {prompt}"

        response = self.model.generate_content(
            full_prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.5,
            )
        )
        return response.text

## Create Engineer Agent Class

In [14]:
@agent(name="engineer")
class EngineerAgent:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-1.5-flash')

    @operation
    def completion(self, prompt: str):
        system_prompt = """You are a software engineer. Your task is to write clean, efficient Python code.
        Only output Python code without any markdown formatting or code blocks.
        Write well-documented, production-ready functions."""

        full_prompt = f"{system_prompt}\n\nUser request: {prompt}"

        response = self.model.generate_content(
            full_prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.5,
            )
        )
        return response.text

## Step 8: Instantiate Agents

In [15]:
qa = QaAgent()
engineer = EngineerAgent()

## Step 9: Generate Code with Engineer Agent

In [16]:
generated_func = engineer.completion("python function to test prime number")

## Step 10: Display Generated Code

In [17]:
display(Markdown("```python\n" + generated_func + "\n```"))

```python
def is_prime(number):
    """
    Checks if a number is a prime number.

    Args:
        number: An integer greater than 1.

    Returns:
        True if the number is prime, False otherwise.  Returns False for numbers less than or equal to 1.
        Raises TypeError if input is not an integer.
    """
    if not isinstance(number, int):
        raise TypeError("Input must be an integer.")
    if number <= 1:
        return False
    if number <= 3:
        return True
    if number % 2 == 0 or number % 3 == 0:
        return False
    i = 5
    while i * i <= number:
        if number % i == 0 or number % (i + 2) == 0:
            return False
        i += 6
    return True

```

## Step 11: Generate Tests with QA Agent

In [18]:
generated_test = qa.completion(
    "Write a python unit test that test the following function: \n " + generated_func
)

## Step 12: Display Generated Tests

In [19]:
display(Markdown("```python\n" + generated_test + "\n```"))

```python
import unittest

def is_prime(number):
    """
    Checks if a number is a prime number.

    Args:
        number: An integer greater than 1.

    Returns:
        True if the number is prime, False otherwise.  Returns False for numbers less than or equal to 1.
        Raises TypeError if input is not an integer.
    """
    if not isinstance(number, int):
        raise TypeError("Input must be an integer.")
    if number <= 1:
        return False
    if number <= 3:
        return True
    if number % 2 == 0 or number % 3 == 0:
        return False
    i = 5
    while i * i <= number:
        if number % i == 0 or number % (i + 2) == 0:
            return False
        i += 6
    return True

class TestIsPrime(unittest.TestCase):
    def test_positive_cases(self):
        self.assertTrue(is_prime(2))
        self.assertTrue(is_prime(3))
        self.assertTrue(is_prime(5))
        self.assertTrue(is_prime(7))
        self.assertTrue(is_prime(11))
        self.assertTrue(is_prime(13))
        self.assertTrue(is_prime(97))
        self.assertTrue(is_prime(101))

    def test_negative_cases(self):
        self.assertFalse(is_prime(1))
        self.assertFalse(is_prime(4))
        self.assertFalse(is_prime(6))
        self.assertFalse(is_prime(8))
        self.assertFalse(is_prime(9))
        self.assertFalse(is_prime(10))
        self.assertFalse(is_prime(100))
        self.assertFalse(is_prime(1000))

    def test_edge_cases(self):
        self.assertTrue(is_prime(3))

    def test_type_error(self):
        with self.assertRaises(TypeError):
            is_prime(3.14)
        with self.assertRaises(TypeError):
            is_prime("10")
        with self.assertRaises(TypeError):
            is_prime([10])

    def test_large_prime(self):
        self.assertTrue(is_prime(997))

    def test_large_composite(self):
        self.assertFalse(is_prime(1000))


if __name__ == '__main__':
    unittest.main()

```

# Traces

Now Go to AgentOps website from where you have take the api and go to Traces Section you can see all the traces of the Agent

<img src="images/Screenshot 2025-07-25 102314.png">

<img src="images/Screenshot 2025-07-25 102351.png">